ver1: 기존에 프로젝트랩에서 하던 대로, 사용금액의 비율% 계산
- data_1901_2004_classified_2_outliered -> 전처리 -> processed_data_1901_2004_ver1

# ---자동화---

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import numpy as np
import unicodedata
import os
import time
from tqdm import tqdm

data_dir = '/content/drive/My Drive/영빈/data/'
data = pd.read_feather(data_dir + 'data_1901_2004_classified_2_outliered.ftr', # 
                       columns=['년', '월', '가맹점소재지1', '가맹점소재지2', '성별', '연령대별', '연평균소득추정','이용금액', '이용건수', '업종']) # index col 빼고 읽어오기


""" --------------- 날짜 루프 ------------------ """


date_list = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912,
             202001, 202002, 202003, 202004]

all_data = []
from tqdm import tqdm
for date in tqdm(date_list):

  year = int(str(date)[:4]) # 2019
  month = str(date)[4:] # 01

  if month[0] == '0':
    month = int(month[1]) # 1
  else:
    month = int(month) 
  
  """ 데이터 월별 필터링 """

  df = data[(data['년']==year) & (data['월']==month)]
  print('월별 데이터: ',df.shape)



  """ 전처리 """



  """ 고객 그룹별 전체 이용금액 """

  grouped = df.groupby(['년', '월', '가맹점소재지1', '가맹점소재지2', '성별','연령대별', '연평균소득추정']) 
  grouped_sum = grouped['이용금액'].sum()
  df_spend_total = pd.DataFrame(grouped_sum)
  #df_spend_total.drop('x', level = 6, axis=0, inplace=True) # 연평균소득추정=x 삭제
  #df_spend_total = df_spend_total.fillna(0) # NaN -> 0
  ##df_spend_total[df_spend_total['이용금액']==0].index # 고소득층에서 이용금액이 0인 경우가 많음 (B8~B10)
  df_spend_total

  """ 고객 그룹별 업종 이용금액 """

  grouped = df.groupby(['년', '월', '가맹점소재지1', '가맹점소재지2', '성별','연령대별', '연평균소득추정', '업종']) 
  grouped_sum = grouped['이용금액'].sum()
  df_spend_part = pd.DataFrame(grouped_sum)
  df_spend_part

  """ 이용금액 % 생성 """

  for index,row in df_spend_part.iterrows():
      
      sort_list = list(index)[:-1] # e.g., (업종제외) 2019년 1월 서울시 노원구 남성 20대 B1
      year, month, region1, region2, gender, age, income = sort_list[0], sort_list[1], sort_list[2], sort_list[3], sort_list[4], sort_list[5], sort_list[6]
      spend_total = df_spend_total.loc[     (df_spend_total.index.get_level_values('년') == year) 
                                          & (df_spend_total.index.get_level_values('월') == month) 
                                          & (df_spend_total.index.get_level_values('가맹점소재지1') == region1)   
                                          & (df_spend_total.index.get_level_values('가맹점소재지2') == region2)
                                          & (df_spend_total.index.get_level_values('성별') == gender) 
                                          & (df_spend_total.index.get_level_values('연령대별') == age)
                                          & (df_spend_total.index.get_level_values('연평균소득추정') == income)]['이용금액']
      
      spend_total = int(spend_total)    
      df_spend_part.at[index, '총 이용금액'] = spend_total
      
  df_spend_part['이용금액%'] = df_spend_part['이용금액'] / df_spend_part['총 이용금액'] # * 100

  """ 데이터셋 형태로 만들기 """

  df_final = df_spend_part.copy()
  df_final.drop(['이용금액', '총 이용금액'], axis=1, inplace=True)
  df_final = df_final.reset_index(level=[0])
  df_final = pd.pivot_table(df_final, values='이용금액%', columns='업종', 
                                    index=['년','월','가맹점소재지1','가맹점소재지2', '성별', '연령대별', '연평균소득추정'])
  df_final = df_final.fillna(0) # NaN -> 0
  df_final["합계%"] = df_final.sum(axis=1)

  """ reverse dict """ 

  # load data
  import pickle
  with open(data_dir + 'dict_pickle/region1_dict.pickle', 'rb') as fr:
      region1_dict = pickle.load(fr)
  with open(data_dir + 'dict_pickle/region2_dict.pickle', 'rb') as fr:
      region2_dict = pickle.load(fr)
  with open(data_dir + 'dict_pickle/gender_dict.pickle', 'rb') as fr:
      gender_dict = pickle.load(fr)
  with open(data_dir + 'dict_pickle/age_dict.pickle', 'rb') as fr:
      age_dict = pickle.load(fr)
  with open(data_dir + 'dict_pickle/income_dict.pickle', 'rb') as fr:
      income_dict = pickle.load(fr)
  with open(data_dir + 'dict_pickle/store_dict.pickle', 'rb') as fr:
      store_dict = pickle.load(fr)

  region1_dict = {value : key for (key, value) in region1_dict.items()}
  region2_dict = {value : key for (key, value) in region2_dict.items()}
  gender_dict = {value : key for (key, value) in gender_dict.items()}
  age_dict = {value : key for (key, value) in age_dict.items()}
  income_dict = {value : key for (key, value) in income_dict.items()}
  store_dict = {value : key for (key, value) in store_dict.items()}

  df_final.rename(index = region1_dict, level=2, inplace=True)
  df_final.rename(index = region2_dict, level=3, inplace=True)
  df_final.rename(index = gender_dict, level=4, inplace=True)
  df_final.rename(index = age_dict, level=5, inplace=True)
  df_final.rename(index = income_dict, level=6, inplace=True)
  df_final.rename(columns = store_dict, inplace = True)
  df_final.reset_index(inplace=True)
  print('최종 데이터: ', df_final.shape)

  all_data.append(df_final)

frame = pd.concat(all_data, axis=0, ignore_index=True)
frame.to_feather(data_dir + 'processed_data_1901_2004_ver1.ftr')

Mounted at /content/drive


  0%|          | 0/16 [00:00<?, ?it/s]

월별 데이터:  (5286559, 10)


  6%|▋         | 1/16 [15:02<3:45:34, 902.32s/it]

최종 데이터:  (20033, 26)
월별 데이터:  (5126968, 10)


...

최종 데이터:  (18499, 26)
월별 데이터:  (4980641, 10)


 94%|█████████▍| 15/16 [3:45:23<12:49, 769.69s/it]

최종 데이터:  (19757, 26)
월별 데이터:  (5255586, 10)


100%|██████████| 16/16 [3:57:12<00:00, 889.54s/it]

최종 데이터:  (20101, 26)


## 결과 확인

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
data_dir = '/content/drive/My Drive/영빈/data/'
df = pd.read_feather(data_dir + 'clustering_data_1901_2004_ver{0}.ftr'.format(1))
df

Mounted at /content/drive


,년,월,가맹점소재지1,가맹점소재지2,성별,연령대별,연평균소득추정,가정용품,개인미용,개인운송,교육서비스,기타,대형판매,보건,보험,숙박서비스,식료품,여객운송,오락_문화,음식점,의류_잡화,전자상거래,정보통신,종합소매,주거,합계%
0,2019,1,강원도,강릉시,남성,20대,B1,0.000000,0.016968,0.026919,0.0,0.002995,0.070005,0.006491,0.000000,0.037965,0.025353,0.000000,0.042588,0.546769,0.009083,0.0,0.000000,0.211340,0.003523,1.0
1,2019,1,강원도,강릉시,남성,20대,B2,0.001245,0.018671,0.050827,0.0,0.006558,0.006698,0.048198,0.000000,0.031228,0.037825,0.000123,0.055453,0.455255,0.030573,0.0,0.000284,0.250780,0.006281,1.0
2,2019,1,강원도,강릉시,남성,20대,B3,0.006838,0.017268,0.076535,0.0,0.003972,0.057428,0.014446,0.000000,0.024132,0.040246,0.003582,0.049992,0.490051,0.018139,0.0,0.000169,0.190898,0.006304,1.0
3,2019,1,강원도,강릉시,남성,20대,B4,0.000217,0.012398,0.085893,0.0,0.006220,0.063568,0.019497,0.000000,0.031590,0.038827,0.002346,0.034838,0.511056,0.012651,0.0,0.000000,0.177576,0.003322,1.0
4,2019,1,강원도,강릉시,남성,20대,B5,0.000000,0.003997,0.012757,0.0,0.003057,0.088494,0.002322,0.000000,0.039834,0.029542,0.000000,0.034749,0.583757,0.015002,0.0,0.000000,0.183910,0.002580,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337029,2020,4,충청북도,충주시,여성,60대 이상,B3,0.029227,0.022472,0.110701,0.0,0.005983,0.084747,0.183780,0.000000,0.003846,0.065866,0.000000,0.080076,0.120656,0.033512,0.0,0.000408,0.243995,0.014730,1.0
337030,2020,4,충청북도,충주시,여성,60대 이상,B4,0.016502,0.022778,0.102699,0.0,0.019791,0.062486,0.162471,0.001317,0.003284,0.063448,0.000000,0.148258,0.136114,0.023608,0.0,0.000167,0.226666,0.010410,1.0
337031,2020,4,충청북도,충주시,여성,60대 이상,B5,0.000000,0.000000,0.063387,0.0,0.000000,0.120413,0.050646,0.000000,0.000000,0.019304,0.000000,0.246296,0.184772,0.049689,0.0,0.000000,0.264008,0.001485,1.0
337032,2020,4,충청북도,충주시,여성,60대 이상,B6,0.000000,0.000000,0.000000,0.0,0.000000,0.201937,0.059451,0.000000,0.000000,0.000000,0.000000,0.453200,0.224740,0.000000,0.0,0.000000,0.060672,0.000000,1.0


## features 평균 그래프

In [ ]:
""" 코랩 한글 """

!pip install matplotlib -U
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 런타임 재시작

In [ ]:
# 코랩 한글
import matplotlib.pyplot as plt
plt.rcParams['font.family']=['NanumGothic', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

import seaborn as sns
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
sns.set_palette("pastel")

features = ['가정용품', '개인미용', '개인운송', '교육서비스', '기타', '대형판매', '보건', '보험', '숙박서비스',
       '식료품', '여객운송', '오락_문화', '음식점', '의류_잡화', '전자상거래', '정보통신', '종합소매', '주거']

means = df[features].mean().sort_values(ascending=False)

plt.rcParams["figure.figsize"] = (10,6)
means.plot(kind='bar')
plt.xticks(rotation=30)
plt.title('features 평균%', size=20)
plt.grid(True, axis='y')
print(means.index)

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import unicodedata
import os
import time
from tqdm import tqdm

data_dir = '/content/drive/My Drive/영빈/data/'
data = pd.read_feather(data_dir + 'data_1901_2004_classified_2_.ftr', # 88656835 rows
                       columns=['년', '월', '가맹점소재지1', '가맹점소재지2', '성별', '연령대별', '연평균소득추정','이용금액', '이용건수', '업종']) # index col 빼고 읽어오기
data.info()

In [ ]:
data.sample(5)

,년,월,가맹점소재지1,가맹점소재지2,성별,연령대별,연평균소득추정,이용금액,이용건수,업종
9943776,2019,4,경기도,광명시,남성,40대,B4,244300.0,14,보건
69768192,2020,1,서울특별시,노원구,여성,50대,B5,73000.0,7,음식점
30216974,2019,4,전라남도,목포시,여성,60대 이상,B4,114000.0,3,오락_문화
57075534,2019,12,부산광역시,부산진구,남성,30대,B3,1238090.0,17,보건
74349484,2020,3,경상북도,칠곡군,여성,30대,B3,436500.0,13,오락_문화


# 데이터 월별 필터링

In [ ]:
""" 2019년 6월 데이터로 연습 """

df = data[(data['년'] == 2020) & (data['월'] == 4)]
print(df.shape)
sorted(df.업종.unique())

(5336937, 10)


['가정용품',
 '개인미용',
 '개인운송',
 '교육서비스',
 '기타',
 '대형판매',
 '보건',
 '보험',
 '숙박서비스',
 '식료품',
 '여객운송',
 '오락_문화',
 '음식점',
 '의류_잡화',
 '전자상거래',
 '정보통신',
 '종합소매',
 '주거']

# 전처리
- col: 업종 20개
  - 식당, 자동차서비스, 보건위생, 식당, 주점 등
- row: 고객 그룹
  - 지역, 성별, 연령대별, 연평균소득추정 
- value: 이용금액(%)

In [ ]:
""" 고객 그룹별 전체 이용금액 """

grouped = df.groupby(['년', '월', '가맹점소재지1', '가맹점소재지2', '성별','연령대별', '연평균소득추정']) # ['년', '월', '가맹점소재지1', '가맹점소재지2', '성별','연령대별', '연평균소득추정']
grouped_sum = grouped['이용금액'].sum()
df_spend_total = pd.DataFrame(grouped_sum)
#df_spend_total.drop('x', level = 6, axis=0, inplace=True) # 연평균소득추정=x 삭제
#df_spend_total = df_spend_total.fillna(0) # NaN -> 0
##df_spend_total[df_spend_total['이용금액']==0].index # 고소득층에서 이용금액이 0인 경우가 많음 (B8~B10)
df_spend_total

이용금액
년    월 가맹점소재지1 가맹점소재지2 성별 연령대별 연평균소득추정             
2020 4 0       3       0  0    0         28671304.0
                               3        480193216.0
                               4        752120896.0
                               5        326631392.0
                               6         22937610.0
...                                             ...
       16      183     1  5    4        828204928.0
                               5        636221184.0
                               6         17235680.0
                               7          1892320.0
                               8            48650.0

[20101 rows x 1 columns]

In [ ]:
""" 고객 그룹별 업종 이용금액 """

grouped = df.groupby(['년', '월', '가맹점소재지1', '가맹점소재지2', '성별','연령대별', '연평균소득추정', '업종']) # ['업종', '년', '월', '가맹점소재지1', '가맹점소재지2', '성별','연령대별', '연평균소득추정']
grouped_sum = grouped['이용금액'].sum()
df_spend_part = pd.DataFrame(grouped_sum)
df_spend_part

이용금액
년    월 가맹점소재지1 가맹점소재지2 성별 연령대별 연평균소득추정 업종           
2020 4 0       3       0  0    0       1    127809.0
                                       2   1356720.0
                                       3   2877000.0
                                       5    164000.0
                                       6    123400.0
...                                              ...
       16      183     1  5    7       3    857600.0
                                       9    425280.0
                                       12   112500.0
                                       16   114810.0
                               8       9     48650.0

[210846 rows x 1 columns]

In [ ]:
""" 이용금액 % 생성 """

from tqdm import tqdm
for index,row in tqdm(df_spend_part.iterrows()):
    
    sort_list = list(index)[:-1] # e.g., (업종제외) 2019년 1월 서울시 노원구 남성 20대 B1
    year, month, region1, region2, gender, age, income = sort_list[0], sort_list[1], sort_list[2], sort_list[3], sort_list[4], sort_list[5], sort_list[6]
    spend_total = df_spend_total.loc[     (df_spend_total.index.get_level_values('년') == year) 
                                        & (df_spend_total.index.get_level_values('월') == month) 
                                        & (df_spend_total.index.get_level_values('가맹점소재지1') == region1)   
                                        & (df_spend_total.index.get_level_values('가맹점소재지2') == region2)
                                        & (df_spend_total.index.get_level_values('성별') == gender) 
                                        & (df_spend_total.index.get_level_values('연령대별') == age)
                                        & (df_spend_total.index.get_level_values('연평균소득추정') == income)]['이용금액']
    
    spend_total = int(spend_total)    
    df_spend_part.at[index, '총 이용금액'] = spend_total
    
df_spend_part['이용금액%'] = df_spend_part['이용금액'] / df_spend_part['총 이용금액'] # * 100
df_spend_part

210846it [13:48, 254.56it/s]


이용금액      총 이용금액     이용금액%
년    월 가맹점소재지1 가맹점소재지2 성별 연령대별 연평균소득추정 업종                                 
2020 4 0       3       0  0    0       1    127809.0  28671304.0  0.004458
                                       2   1356720.0  28671304.0  0.047320
                                       3   2877000.0  28671304.0  0.100344
                                       5    164000.0  28671304.0  0.005720
                                       6    123400.0  28671304.0  0.004304
...                                              ...         ...       ...
       16      183     1  5    7       3    857600.0   1892320.0  0.453200
                                       9    425280.0   1892320.0  0.224740
                                       12   112500.0   1892320.0  0.059451
                                       16   114810.0   1892320.0  0.060672
                               8       9     48650.0     48650.0  1.000000

[210846 rows x 3 columns]

In [ ]:
""" 데이터셋 형태로 만들기 """

df_final = df_spend_part.copy()
df_final.drop(['이용금액', '총 이용금액'], axis=1, inplace=True)
df_final = df_final.reset_index(level=[0])
df_final = pd.pivot_table(df_final, values='이용금액%', columns='업종', 
                                   index=['년','월','가맹점소재지1','가맹점소재지2', '성별', '연령대별', '연평균소득추정'])
df_final = df_final.fillna(0) # NaN -> 0
df_final["합계%"] = df_final.sum(axis=1)
df_final

업종                                             0         1  ...   19  합계%
년    월 가맹점소재지1 가맹점소재지2 성별 연령대별 연평균소득추정                      ...          
2019 6 0       3       0  0    0        0.000000  0.000409  ...  0.0  1.0
                               1        0.000000  0.001626  ...  0.0  1.0
                               2        0.000000  0.000000  ...  0.0  1.0
                               3        0.000841  0.005684  ...  0.0  1.0
                               4        0.002291  0.007227  ...  0.0  1.0
...                                          ...       ...  ...  ...  ...
       16      183     1  5    5        0.022630  0.025637  ...  0.0  1.0
                               6        0.000000  0.000000  ...  0.0  1.0
                               7        0.000000  0.000000  ...  0.0  1.0
                               8        0.000000  0.000000  ...  0.0  1.0
                               9        0.000000  0.000000  ...  0.0  1.0

[22233 rows x 21 columns]

In [ ]:
""" reverse dict """ 

# load data
import pickle
with open(data_dir + 'dict_pickle/region1_dict.pickle', 'rb') as fr:
    region1_dict = pickle.load(fr)
with open(data_dir + 'dict_pickle/region2_dict.pickle', 'rb') as fr:
    region2_dict = pickle.load(fr)
with open(data_dir + 'dict_pickle/gender_dict.pickle', 'rb') as fr:
    gender_dict = pickle.load(fr)
with open(data_dir + 'dict_pickle/age_dict.pickle', 'rb') as fr:
    age_dict = pickle.load(fr)
with open(data_dir + 'dict_pickle/income_dict.pickle', 'rb') as fr:
    income_dict = pickle.load(fr)
with open(data_dir + 'dict_pickle/store_dict.pickle', 'rb') as fr:
    store_dict = pickle.load(fr)

region1_dict = {value : key for (key, value) in region1_dict.items()}
region2_dict = {value : key for (key, value) in region2_dict.items()}
gender_dict = {value : key for (key, value) in gender_dict.items()}
age_dict = {value : key for (key, value) in age_dict.items()}
income_dict = {value : key for (key, value) in income_dict.items()}
store_dict = {value : key for (key, value) in store_dict.items()}


df_final.rename(index = region1_dict, level=2, inplace=True)
df_final.rename(index = region2_dict, level=3, inplace=True)
df_final.rename(index = gender_dict, level=4, inplace=True)
df_final.rename(index = age_dict, level=5, inplace=True)
df_final.rename(index = income_dict, level=6, inplace=True)
df_final.rename(columns = store_dict, inplace = True)
df_final.reset_index(inplace=True)
df_final

업종,년,월,가맹점소재지1,가맹점소재지2,성별,연령대별,연평균소득추정,"가구,가전",기타,대형판매,레져,"문화,취미","보건,위생",서비스,"서적,문구",쇼핑,식당,식료품,"여행,숙박","의료,보험",인테리어,자동차,자동차서비스,전자상거래,종합소매,주점,학원,합계%
0,2019,6,강원도,강릉시,남성,20대,B1,0.000000,0.000409,0.026981,0.092069,0.010510,0.025194,0.003804,0.006999,0.011882,0.371776,0.021731,0.019434,0.007382,0.000000,0.000000,0.013528,0.0,0.268119,0.120184,0.0,1.0
1,2019,6,강원도,강릉시,남성,20대,B10,0.000000,0.001626,0.046908,0.034297,0.005871,0.016539,0.004568,0.004995,0.011336,0.449621,0.030523,0.009921,0.011552,0.000000,0.000000,0.029432,0.0,0.256918,0.085895,0.0,1.0
2,2019,6,강원도,강릉시,남성,20대,B11,0.000000,0.000000,0.047943,0.021378,0.020605,0.000000,0.000000,0.000000,0.000000,0.459788,0.000000,0.183295,0.000000,0.000000,0.000000,0.040608,0.0,0.226382,0.000000,0.0,1.0
3,2019,6,강원도,강릉시,남성,20대,B2,0.000841,0.005684,0.045066,0.062950,0.008047,0.020918,0.008066,0.006349,0.018941,0.362040,0.029147,0.024253,0.017779,0.000924,0.000508,0.040771,0.0,0.248362,0.099356,0.0,1.0
4,2019,6,강원도,강릉시,남성,20대,B3,0.002291,0.007227,0.049237,0.053395,0.006728,0.018487,0.008690,0.004410,0.015958,0.389727,0.034081,0.034530,0.027447,0.000180,0.000000,0.065844,0.0,0.201147,0.080620,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22228,2019,6,충청북도,충주시,여성,60대 이상,B4,0.022630,0.025637,0.070306,0.116088,0.003082,0.018768,0.004653,0.000990,0.050176,0.156760,0.049196,0.013314,0.129890,0.003787,0.000000,0.124256,0.0,0.199170,0.011297,0.0,1.0
22229,2019,6,충청북도,충주시,여성,60대 이상,B5,0.000000,0.000000,0.081546,0.275388,0.000000,0.000000,0.000000,0.000000,0.005684,0.278295,0.018857,0.000000,0.152774,0.000000,0.000000,0.050624,0.0,0.135264,0.001568,0.0,1.0
22230,2019,6,충청북도,충주시,여성,60대 이상,B6,0.000000,0.000000,0.101534,0.252972,0.000000,0.000000,0.000000,0.000000,0.000000,0.579036,0.000000,0.000000,0.000000,0.000000,0.000000,0.053221,0.0,0.000000,0.013237,0.0,1.0
22231,2019,6,충청북도,충주시,여성,60대 이상,B7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1.0
